In [89]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

In [90]:
import matplotlib
from keras.models import * 
from keras.layers import *
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"C:\Users\me\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"C:\Users\me\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level1_Patient Allergens")

In [91]:
# Confirm that patient sheet imported correctly
# Comment below line before committing
# patients.head()

In [92]:
# Confirm allergy sheet imported correctly
# Comment line before committing
# allergies.head()

In [93]:
# Merge columns by ID if needed
patientAllergies = patients.merge(allergies, on = "SFM Id")
# Comment line before committing
# patientAllergies.head()

In [94]:
# Drop ID and location columns from dataframe
patientsTrimmed = patients.drop(['SFM Id', 'City', 'State', 'Country'], axis = 1)

In [95]:
# One-hot encode Gender column
patientsTrimmed['Gender'] = pd.Categorical(patientsTrimmed['Gender'].str.strip())
gender_onehot = pd.get_dummies(patientsTrimmed['Gender'], prefix = "Gender",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('Gender', axis = 1)
patientsTrimmed = patientsTrimmed.join(gender_onehot)

In [96]:
# One-hot encode SkinTone column
patientsTrimmed['SkinTone'] = pd.Categorical(patientsTrimmed['SkinTone'].str.strip())
skintone_onehot = pd.get_dummies(patientsTrimmed['SkinTone'], prefix = "SkinTone",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('SkinTone', axis = 1)
patientsTrimmed = patientsTrimmed.join(skintone_onehot)

In [97]:
# One-hot encode FitzPatrickSkinPhotoType column
patientsTrimmed['FitzpatrickSkinPhotoType'] = pd.Categorical(patientsTrimmed['FitzpatrickSkinPhotoType'].str.strip())
# Dropping first here since it is a blank variable in the column
fitzpatrick_onehot = pd.get_dummies(patientsTrimmed['FitzpatrickSkinPhotoType'], prefix = "Fitzpatrick",
                                    prefix_sep = "-", drop_first = True, dtype = int)
patientsTrimmed = patientsTrimmed.drop('FitzpatrickSkinPhotoType', axis = 1)
patientsTrimmed = patientsTrimmed.join(fitzpatrick_onehot)

In [98]:
# Switching to TextVectorization (Tokenizer is deprecated)
from keras.layers import TextVectorization
# Replace commas with whitespace
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].str.replace(',', ' ')
# Set the max length based on whitespace characters
max_len = patientsTrimmed['SkinConditions'].str.count(' ').max()
# Create TextVectorization object, separating on whitespace and using the max_len from earlier
vectorizer = TextVectorization(split = 'whitespace', output_sequence_length = max_len)

In [99]:
# Adapt using the column we want to convert
vectorizer.adapt(patientsTrimmed['SkinConditions'].values)
# Reset the column after converting values to vector and placing in array
skinConditions = vectorizer(patientsTrimmed['SkinConditions']).numpy()
patientsTrimmed = patientsTrimmed.drop('SkinConditions', axis = 1)

In [100]:
patientsArr = patientsTrimmed.values
input_data = np.concatenate((patientsArr, skinConditions), axis = 1)

In [ ]:
''' Old code for SkinConditions conversion - leaving in case we need later
# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text
## Delete if this idea does not work
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Flatten entries into actual lists
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].apply(lambda x: ' '.join(x))
# Tokenize the characters of the columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patientsTrimmed['SkinConditions'])
# Create and pad sequences of the tokens
patientsTrimmed['SkinConditions'] = tokenizer.texts_to_sequences(patientsTrimmed['SkinConditions'])
max_length = max(len(seq) for seq in patientsTrimmed['SkinConditions'])
testVar = pad_sequences(patientsTrimmed['SkinConditions'], maxlen = max_length, padding = "post")
patientsTrimmed['SkinConditions'] = testVar.tolist()
'''

In [102]:
# Drop ID column for preprocessing - ID should have no effect on prediction
allergiesNoId = allergies.drop('SFM Id', axis = 1)
# Remove all non-digit characters, then replace empty cells with NaN
allergiesNoId = allergiesNoId.replace(r'\D+', '', regex = True).replace('', np.nan)
# Set all NaN cells to 0
allergiesNoId = allergiesNoId.fillna(0)
# Convert entire dataframe to integer
allergiesNoId = allergiesNoId.astype(int)

In [103]:
allergiesNoId['AllergiesList'] = allergiesNoId.astype(str).apply(' '.join, axis=1)
allergiesNoId['AllergiesList'] = allergiesNoId['AllergiesList'].str.split()

In [104]:
# TODO: create a training/testing split for the data

In [106]:
from sklearn.preprocessing import MultiLabelBinarizer
# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

allergiesArray = np.array(allergiesNoId['AllergiesList'])
# One-hot encode data
allergiesArray = mlb.fit_transform(allergiesArray)

In [109]:
# TODO: Refine layers/sequences to actually make the model once we get it loading in correctly
# classes should be total number of outputs (1 for each ingredient)
n_classes = 731

# Input layer
input_shape = (40,)
inputs = keras.Input(input_shape)

# Hidden layers
x = Dense(128, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)

# Output layer - use multilabel classification
predictions = Dense(n_classes, activation='softmax')(x)

#Tie model together
model = keras.Model(inputs = inputs, outputs = predictions)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [110]:
history = model.fit(
   input_data, allergiesArray, 
   batch_size = 128, 
   epochs = 20, 
   verbose = 1
)

Epoch 1/20

78/78 [==============================] - 2s 9ms/step - loss: 1.7310 - accuracy: 0.8190
Epoch 2/20
78/78 [==============================] - 1s 8ms/step - loss: 0.0466 - accuracy: 1.0000
Epoch 3/20
78/78 [==============================] - 1s 8ms/step - loss: 0.0398 - accuracy: 1.0000
Epoch 4/20
78/78 [==============================] - 1s 8ms/step - loss: 0.0384 - accuracy: 1.0000
Epoch 5/20
78/78 [==============================] - 1s 9ms/step - loss: 0.0378 - accuracy: 1.0000
Epoch 6/20
78/78 [==============================] - 1s 8ms/step - loss: 0.0376 - accuracy: 1.0000
Epoch 7/20
78/78 [==============================] - 1s 8ms/step - loss: 0.0375 - accuracy: 1.0000
Epoch 8/20
78/78 [==============================] - 1s 7ms/step - loss: 0.0374 - accuracy: 1.0000
Epoch 9/20
78/78 [==============================] - 1s 7ms/step - loss: 0.0375 - accuracy: 1.0000
Epoch 10/20
78/78 [==============================] - 1s 8ms/step - loss: 0.0374 - accuracy: 1.0000
Epoch 11/20
78/78 

In [ ]:
# Get the average prediction score across all epochs
score = model.evaluate(x_test, y_test, verbose = 0)
score #First number is loss, second is accuracy

In [ ]:
# TODO: Save Keras model as separate file